# UTAG on MIBI data

## Load libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns
from utag import utag

from utag.segmentation import evaluate_performance

from utag.utils import celltype_connectivity, domain_connectivity
'''
from utag.visualize import (
    add_scale_box_to_fig,
    add_spatial_image,
    adj2chord,
    draw_network,
)
'''
sc.settings.set_figure_params(dpi=100, dpi_save=300, fontsize=6)
import matplotlib
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["axes.grid"] = False

In [ ]:
import pickle as pkl

## Load and format data

In [ ]:
data = pd.read_csv("cell_table_size_normalized.csv")

In [ ]:
# Adapt with the columns you want to use to describe each cell
cellfeatures = ['ASCT2', 'ATP5A', 'CD14', 'CD163', 'CD20', 'CD31', 'CD36',
               'CD3e', 'CD4', 'CD45', 'CD45RO', 'CD56', 'CD68', 'CD8', 'CD98',
               'COL1A1', 'CPT1A', 'CS', 'Calprotectin', 'Caveolin', 'ChyTry', 'CytC',
               'EpCAM', 'FoxP3', 'G6PD', 'GLS', 'GLUT1', 'HLADR', 'IL17A', 'Ki67',
               'LDH', 'MCT1', 'PD1', 'PDL1', 'PKM2', 'PanCK', 'SMA', 'Vimentin']

In [ ]:
# Drop brain samples
data = data[[x[-4:] != "Brai" for x in data["fov"]]]

In [ ]:
adata = sc.AnnData(data.loc[:,cellfeatures])

In [ ]:
# Annotation: image for each cell
adata.obs["fov"] = pd.Categorical(data["fov"])  # Categoricals are preferred for efficiency

In [ ]:
# Annotation: position for each cell
adata.obsm["spatial"] = data[["centroid-0","centroid-1"]].values

In [ ]:
%%time
results = utag(adata, 
               slide_key="fov",
               max_dist=50,
               normalization_mode='l1_norm',
               apply_clustering=True,
               clustering_method = 'leiden', 
               resolutions = [0.02,0.05,0.1])

In [ ]:
# To compare FOVs, we want the cluster colors to be consistent so we define
# a palette
niches = results.obs['UTAG Label_leiden_0.1'].unique()
shared_palette = {n: matplotlib.colormaps['tab20'](int(n)) for n in niches}

In [ ]:
for roi in results.obs['fov'].unique():
    result = results[results.obs['fov'] == roi].copy()
    sc.pl.spatial(result,
                  color = 'UTAG Label_leiden_0.1',
                  palette = shared_palette,
                  spot_size = 15, title = roi)

## Metabolic niches
Discard lineage markers

In [ ]:
metabolicfeatures = ['ASCT2', 'ATP5A', 'CD36',
                   'CD98',
                   'CPT1A', 'CS', 'CytC',
                   'G6PD', 'GLS', 'GLUT1',
                   'LDH', 'MCT1','PKM2']

In [ ]:
adata = sc.AnnData(data.loc[:,metabolicfeatures])

In [ ]:
# Annotation: image for each cell
adata.obs["fov"] = pd.Categorical(data["fov"])  # Categoricals are preferred for efficiency

In [ ]:
# Annotation: position for each cell
adata.obsm["spatial"] = data[["centroid-0","centroid-1"]].values

In [ ]:
%%time
results = utag(adata, 
               slide_key="fov",
               max_dist=50,
               normalization_mode='l1_norm',
               apply_clustering=True,
               clustering_method = 'leiden', 
               resolutions = [0.02,0.05,0.1])

In [ ]:
for roi in results.obs['fov'].unique():
    result = results[results.obs['fov'] == roi].copy()
    sc.pl.spatial(result,
                  color = 'UTAG Label_leiden_0.1',
                  palette = shared_palette,
                  spot_size = 15, title = roi)